In [1]:
    import requests
    import pandas as pd
    from datetime import date

    # URL template (corrected)
    url = "https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15"

    # Instrument tokens
    tickerData = {"NIFTY": {"id": 256265}, "BANKNIFTY": {"id": 260105}}

    # Interval mapping
    KI = {
        "3m": "3minute",
        "5m": "5minute",
        "15m": "15minute",
        "60m": "60minute",
        "day": "day",
    }

    # Function to get candle data
    def getCandles(symbol, fromDate, toDate, timeframe):
        headers = {
            "authorization": "enctoken l9RBs2mUaCOtHh76/Al766xp7yJ1ndunq9zE2PslZvpMEXu0ccRXlWhWHNUOyJX19msbiNnAcd37PdO0yb1ikRiT3q1N7v2hutAK2FB3s2GAGxkfVgusyQ=="
            # Replace this with your actual token securely
        }
        curUrl = url.format(tickerData[symbol]["id"], timeframe, fromDate, toDate)
        print("Requesting:", curUrl)

        try:
            session = requests.session()
            response = session.get(curUrl, headers=headers)
            r = response.json()
        except Exception as error:
            print("API request failed:", error)
            return pd.DataFrame()

        try:
            history = r["data"]["candles"]
            history_df = pd.DataFrame(history)
            history_df = history_df.rename(
                columns={
                    0: "Datetime",
                    1: "open",
                    2: "high",
                    3: "low",
                    4: "close",
                    5: "volume",
                }
            )
            history_df["Datetime"] = pd.to_datetime(history_df["Datetime"])
            history_df.set_index("Datetime", inplace=True)
            return history_df
        except Exception as error:
            print("Data processing failed:", error)
            return pd.DataFrame()

    # Step 3: MACD Function
    def MACD(DF, len1, len2, len3):
        """
        Function to calculate MACD
        len1 = Fast EMA (typically 12)
        len2 = Slow EMA (typically 26)
        len3 = Signal EMA (typically 9)
        """
        df = DF.copy()
        df["MA_Fast"] = df["close"].ewm(span=len1, min_periods=len1).mean()
        df["MA_Slow"] = df["close"].ewm(span=len2, min_periods=len2).mean()
        df["MACD"] = df["MA_Fast"] - df["MA_Slow"]
        df["Signal"] = df["MACD"].ewm(span=len3, min_periods=len3).mean()
        df.dropna(inplace=True)
        return df

        

    print("Start")
    symbol_data = getCandles("NIFTY", "2025-03-20", "2025-03-25", KI["15m"])


    if not symbol_data.empty:
        MACDdata = MACD(symbol_data, 12, 26, 9)
        print(MACDdata)
    else:
        print("No OHLC data available.")


Start
Requesting: https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15
                               open      high       low     close  volume  6  \
Datetime                                                                       
2025-06-11 12:00:00+05:30  25210.15  25213.00  25205.00  25211.90       0  0   
2025-06-11 12:05:00+05:30  25211.60  25213.25  25208.65  25209.90       0  0   
2025-06-11 12:10:00+05:30  25209.90  25213.20  25204.05  25205.95       0  0   
2025-06-11 12:15:00+05:30  25206.20  25209.15  25201.50  25207.15       0  0   
2025-06-11 12:20:00+05:30  25206.20  25220.40  25205.00  25214.80       0  0   
...                             ...       ...       ...       ...     ... ..   
2025-06-13 15:05:00+05:30  24702.60  24727.90  24702.60  24727.90       0  0   
2025-06-13 15:10:00+05:30  24727.95  24728.45  24717.20  24725.60       0  0   
2025-06-13 15:15:00+05:30  24725.15  24725.90  24709.75  24717.95